# Proyecto Geo Espaciales

**VER PROBLEMA DE QUE ESTAMOS CONSIDERANDO CUERPO DE BOMBEROS**

## Integrantes:
- Camila Basulto
- Benjamín Urbina

## Tema:
- Análisis de cobertura de servicios existentes
- Identificación de zonas aptas para construir

Haremos un mapa con zonas aptas para urbanizar, con recomendaciones específicas sobre qué tipo de infraestructura construir en cada una.


# Preprocesamiento de datos

### Carga de datos raw

In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
import geemap,ee
import folium
import folium

Se consideran las siguientes ciudades para el análisis:
**ARICA, LA SERENA, VIÑA DEL MAR, VALPARAÍSO, CONCEPCIÓN, RANCAGUA, PUERTO MONTT, VALDIVIA**

# Áreas de interés (A partir de Limites Urbanos Censales)

Se realizara un filtrado espacial de los registros de servicios (bomberos, carabineros, salud y educación) utilizando las áreas de interés (AOI) correspondientes a las zonas urbanas de cada ciudad. Esto fue necesario ya que, sin este filtro, algunos establecimientos aparecían ubicados en zonas alejadas o poco representativas, como islas o sectores rurales distantes, que no corresponden al núcleo urbano de análisis.

In [2]:
r8 = gpd.read_file("data/cartografia/r08/Limite_Urbano_Censal.shp")
r8.head()

,CUT,N_REGION,N_PROVINCI,N_COMUNA,N_URBANO,TIPO,CATEGORIA,SHAPE_Leng,SHAPE_Area,geometry
0,8101,BIOBÍO,CONCEPCIÓN,CONCEPCIÓN,CONCEPCIÓN,CAPITAL REGIONAL,CIUDAD,0.452071,0.004314,"POLYGON ((-73.04158 -36.77652, -73.04156 -36.7..."
1,8102,BIOBÍO,CONCEPCIÓN,CORONEL,CORONEL,CAPITAL COMUNAL,CIUDAD,0.399893,0.004159,"POLYGON ((-73.14777 -36.92963, -73.14749 -36.9..."
2,8103,BIOBÍO,CONCEPCIÓN,CHIGUAYANTE,CHIGUAYANTE,CAPITAL COMUNAL,CIUDAD,0.321383,0.001858,"POLYGON ((-73.03787 -36.86095, -73.03784 -36.8..."
3,8104,BIOBÍO,CONCEPCIÓN,FLORIDA,FLORIDA,CAPITAL COMUNAL,PUEBLO,0.123348,0.000265,"POLYGON ((-72.64563 -36.81435, -72.64558 -36.8..."
4,8105,BIOBÍO,CONCEPCIÓN,HUALQUI,HUALQUI,CAPITAL COMUNAL,CIUDAD,0.178118,0.000794,"POLYGON ((-72.94737 -36.96682, -72.94718 -36.9..."


In [3]:
conce = gpd.read_file("data/cartografia/r08/Limite_Urbano_Censal.shp")
conce['N_COMUNA'] = conce['N_COMUNA'].str.upper()
aoi_conce = conce[conce['N_COMUNA'] == 'CONCEPCIÓN'].dissolve(by='N_COMUNA')

m_conce = geemap.Map(center=aoi_conce.geometry.centroid.iloc[0].coords[0][::-1], zoom=12)
m_conce.add_gdf(aoi_conce, layer_name='AOI - CONCEPCIÓN')
m_conce

C:\Users\burbi\AppData\Local\Temp\ipykernel_5352\1756477344.py:5: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  m_conce = geemap.Map(center=aoi_conce.geometry.centroid.iloc[0].coords[0][::-1], zoom=12)


Map(center=[-36.81489568131209, -73.03525569102234], controls=(WidgetControl(options=['position', 'transparent…

In [4]:
la_serena = gpd.read_file("data/cartografia/r04/Limite_Urbano_Censal.shp")
la_serena['N_COMUNA'] = la_serena['N_COMUNA'].str.upper()
aoi_serena = la_serena[la_serena['N_COMUNA'] == 'LA SERENA'].dissolve(by='N_COMUNA')

m_serena = geemap.Map(center=aoi_serena.geometry.centroid.iloc[0].coords[0][::-1], zoom=12)
m_serena.add_gdf(aoi_serena, layer_name='AOI - LA SERENA')
m_serena

Map(center=[-29.91596713752511, -71.24032462513384], controls=(WidgetControl(options=['position', 'transparent…

### Hacemos lo mismo con las demás ciudades

In [5]:
# VIÑA DEL MAR
vina = gpd.read_file("data/cartografia/r05/Limite_Urbano_Censal.shp")
vina['N_COMUNA'] = vina['N_COMUNA'].str.upper()
aoi_vina = vina[vina['N_COMUNA'] == 'VIÑA DEL MAR'].dissolve(by='N_COMUNA')

# VALPARAÍSO
valpo = gpd.read_file("data/cartografia/r05/Limite_Urbano_Censal.shp")
valpo['N_COMUNA'] = valpo['N_COMUNA'].str.upper()
aoi_valpo = valpo[valpo['N_COMUNA'] == 'VALPARAÍSO'].dissolve(by='N_COMUNA')

# Arica
arica = gpd.read_file("data/cartografia/r15/Limite_Urbano_Censal.shp")
arica['N_COMUNA'] = arica['N_COMUNA'].str.upper()
aoi_arica = arica[arica['N_COMUNA'] == 'ARICA'].dissolve(by='N_COMUNA')

# RANCAGUA
rancagua = gpd.read_file("data/cartografia/r06/Limite_Urbano_Censal.shp")
rancagua['N_COMUNA'] = rancagua['N_COMUNA'].str.upper()
aoi_rancagua = rancagua[rancagua['N_COMUNA'] == 'RANCAGUA'].dissolve(by='N_COMUNA')

# PUERTO MONTT
pmontt = gpd.read_file("data/cartografia/r10/Limite_Urbano_Censal.shp")
pmontt['N_COMUNA'] = pmontt['N_COMUNA'].str.upper()
aoi_pmontt = pmontt[pmontt['N_COMUNA'] == 'PUERTO MONTT'].dissolve(by='N_COMUNA')

# VALDIVIA
valdivia = gpd.read_file("data/cartografia/r14/Limite_Urbano_Censal.shp")
valdivia['N_COMUNA'] = valdivia['N_COMUNA'].str.upper()
aoi_valdivia = valdivia[valdivia['N_COMUNA'] == 'VALDIVIA'].dissolve(by='N_COMUNA')

---

# Leemos los datos de los servicios

In [6]:
bomberos_df = gpd.read_file('data/bomberos/layer_cuerpos_de_bomberos_20231110074853.shp')
carabineros_df = gpd.read_file('data/carabineros/layer_cuarteles_20220309024623.shp')
salud_df = gpd.read_file('data/salud/layer_establecimientos_de_salud_20240712091725.shp')
educacion_df = gpd.read_file('data/educacion/layer_establecimientos_educacion_escolar_20220309024120.shp')
parvularia_df = gpd.read_file('data/educacion/layer_establecimientos_educacion_parvularia_20220309024143.shp')

In [7]:
bomberos_df.head()

,OBJECTID,NOMBRE_DEL,TIPO_DE_VL,NUMERACIó,NOMBRE_DE,COMUNA,PROVINCIA,REGIóN,FID_CL_COM,ID_2002,NOMBRE,FID_CL_REG,ID_2002_20,NOMBRE_1,geometry
0,308.0,Puerto Williams,Calle,220,O Higgins,Cabo de Hornos (Ex Navarino),Antartica Chilena,Región de Magallanes y la Antártica Chilena,332.0,12201,Cabo de Hornos,14.0,12,Magallanes y de la Antartica Chileno,POINT (-67.60351 -54.93455)
1,306.0,Puerto Porvenir,Calle,80,Bernardo Phillipi,Porvenir,Tierra del Fuego,Región de Magallanes y la Antártica Chilena,331.0,12301,Porvenir,14.0,12,Magallanes y de la Antartica Chileno,POINT (-70.36875 -53.29513)
2,305.0,Punta Arenas,None,824,Chiloé,Punta Arenas,Magallanes,Región de Magallanes y la Antártica Chilena,340.0,12101,Punta Arenas,14.0,12,Magallanes y de la Antartica Chileno,POINT (-70.91018 -53.16251)
3,307.0,Puerto Natales,Calle,1455,Simón Bolivar,Natales,Última Esperanza,Región de Magallanes y la Antártica Chilena,333.0,12401,Puerto Natales,14.0,12,Magallanes y de la Antartica Chileno,POINT (-72.48352 -51.73077)
4,303.0,Cochrane,Avenida,360,Bernardo O Higgins,Cochrane,Capitan Prat,Región de Aysén del General Carlos Ibañez del ...,336.0,11301,Cochrane,13.0,11,Aisen del Gral. Carlos IbaOez del Campo,POINT (-72.57934 -47.25521)


In [8]:
bomberos_df = bomberos_df[['NOMBRE_DEL','COMUNA', 'geometry']]
bomberos_df.reset_index(drop=True, inplace=True)
bomberos_df.rename(columns={'NOMBRE_DEL': 'NOMBRE_CUERPO'}, inplace=True)
bomberos_df.head()

,NOMBRE_CUERPO,COMUNA,geometry
0,Puerto Williams,Cabo de Hornos (Ex Navarino),POINT (-67.60351 -54.93455)
1,Puerto Porvenir,Porvenir,POINT (-70.36875 -53.29513)
2,Punta Arenas,Punta Arenas,POINT (-70.91018 -53.16251)
3,Puerto Natales,Natales,POINT (-72.48352 -51.73077)
4,Cochrane,Cochrane,POINT (-72.57934 -47.25521)


In [9]:
carabineros_df.head()

,NOMBRE_REG,NOMBRE_PRO,NOMBRE_COM,NOMBRE_UNI,TIPO_DE_UN,TIPO_DE_VI,NOMBRE_DE,NUMERO,POINT_X,POINT_Y,HECHO,RG,PREFECTURA,TIPO,ZONA,UNI_COD,UNID_ENCRI,geometry
0,REGION ARICA Y PARINACOTA,ARICA,ARICA,1RA. COM. ARICA,COMISARIA,AVENIDA,SANTA MARIA,900,-70.311605,-18.478186,2014,15.0,ARICA,None,ARICA Y PARINACOTA,15001,4e007229b70f090dd4f3da67df0dad03,POINT Z (-70.3116 -18.47819 0)
1,REGION ARICA Y PARINACOTA,PARINACOTA,PUTRE,2DA. COM. PUTRE,COMISARIA,CALLE,RIQUELME,298,-69.559351,-18.195069,2014,15.0,ARICA,None,ARICA Y PARINACOTA,15002,19b6f5e29f49c81be5ee2950a58c8ede,POINT Z (-69.55935 -18.19507 0)
2,REGION ARICA Y PARINACOTA,ARICA,ARICA,3RA. COM. ARICA,COMISARIA,AVENIDA,AVENIDA LOA,1475,-70.293810,-18.470430,2014,15.0,ARICA,None,ARICA Y PARINACOTA,15003,bf93f1772d2469dbd77638b6ce784178,POINT Z (-70.29381 -18.47043 0)
3,REGION ARICA Y PARINACOTA,PARINACOTA,GENERAL LAGOS,RETEN ALCERRECA (F),RETEN,RUTA,RUTA A-23,KM 40,-69.660406,-17.992306,2014,15.0,ARICA,None,ARICA Y PARINACOTA,15004,None,POINT Z (-69.66041 -17.99231 0)
4,REGION ARICA Y PARINACOTA,PARINACOTA,PUTRE,RETEN CAQUENA (F),RETEN,RUTA,RUTA A-159,KM 12,-69.201796,-18.054583,2014,15.0,ARICA,None,ARICA Y PARINACOTA,15005,None,POINT Z (-69.2018 -18.05458 0)


In [10]:
carabineros_df = carabineros_df[['NOMBRE_UNI','NOMBRE_COM', 'geometry']]
carabineros_df.reset_index(drop=True, inplace=True)
carabineros_df.rename(columns={'NOMBRE_UNI': 'NOMBRE_ESTABLECIMIENTO', 'NOMBRE_COM': 'COMUNA'}, inplace=True)
carabineros_df.head()

,NOMBRE_ESTABLECIMIENTO,COMUNA,geometry
0,1RA. COM. ARICA,ARICA,POINT Z (-70.3116 -18.47819 0)
1,2DA. COM. PUTRE,PUTRE,POINT Z (-69.55935 -18.19507 0)
2,3RA. COM. ARICA,ARICA,POINT Z (-70.29381 -18.47043 0)
3,RETEN ALCERRECA (F),GENERAL LAGOS,POINT Z (-69.66041 -17.99231 0)
4,RETEN CAQUENA (F),PUTRE,POINT Z (-69.2018 -18.05458 0)


In [11]:
salud_df.head()

,COD_ANT,COD_VIG,COD_M_ANT,COD_M_NUEV,COD_REG,NOM_REG,COD_DEP,DEPENDENC,PERTENENCI,TIPO,...,COMPLEJIDA,TIPO_ATEN,F_INCR,CUT_REG,CUT_PROV,CUT_COM,REGION,PROVINCIA,COMUNA,geometry
0,02-010,102010.0,None,None,1.0,REGION DE TARAPACA,2.0,SERVICIO DE SALUD TARAPACA,PERTENECIENTE AL SISTEMA NACIONAL DE SERVICIOS...,DIRECCION SERVICIO DE SALUD,...,BAJA COMPLEJIDAD,ATENCION ABIERTA-AMBULATORIA,None,01,011,01101,TARAPACA,IQUIQUE,IQUIQUE,POINT (-70.15232 -20.21512)
1,02-011,102011.0,02-010,102010,1.0,REGION DE TARAPACA,2.0,SERVICIO DE SALUD TARAPACA,PERTENECIENTE AL SISTEMA NACIONAL DE SERVICIOS...,PROGRAMA DE REPARACION Y ATENCION INTEGRAL DE ...,...,BAJA COMPLEJIDAD,ATENCION ABIERTA-AMBULATORIA,None,01,011,01101,TARAPACA,IQUIQUE,IQUIQUE,POINT (-70.15232 -20.21512)
2,02-030,102030.0,None,None,1.0,REGION DE TARAPACA,2.0,SERVICIO DE SALUD TARAPACA,PERTENECIENTE AL SISTEMA NACIONAL DE SERVICIOS...,UNIDAD DE SALUD FUNCIONARIOS,...,BAJA COMPLEJIDAD,ATENCION ABIERTA-AMBULATORIA,None,01,011,01101,TARAPACA,IQUIQUE,IQUIQUE,POINT (-70.15232 -20.21512)
3,02-100,102100.0,None,None,1.0,REGION DE TARAPACA,2.0,SERVICIO DE SALUD TARAPACA,PERTENECIENTE AL SISTEMA NACIONAL DE SERVICIOS...,HOSPITAL,...,ALTA COMPLEJIDAD,ATENCION CERRADA-HOSPITALARIA,None,01,011,01101,TARAPACA,IQUIQUE,IQUIQUE,POINT (-70.13893 -20.21435)
4,02-200,102200.0,None,None,1.0,REGION DE TARAPACA,1.0,SEREMI DE TARAPACA,NO PERTENECIENTE AL SISTEMA NACIONAL DE SERVIC...,CLINICA,...,MEDIANA COMPLEJIDAD,ATENCION CERRADA-HOSPITALARIA,None,01,011,01101,TARAPACA,IQUIQUE,IQUIQUE,POINT (-70.15497 -20.2183)


In [12]:
salud_df = salud_df[['NOMBRE','TIPO', 'COMUNA', 'geometry']]
salud_df.reset_index(drop=True, inplace=True)
salud_df.rename(columns={'NOMBRE': 'NOMBRE_ESTABLECIMIENTO', 'TIPO': 'TIPO_ESTABLECIMIENTO'}, inplace=True)
salud_df.head()

,NOMBRE_ESTABLECIMIENTO,TIPO_ESTABLECIMIENTO,COMUNA,geometry
0,ACTIVIDADES GESTIONADAS POR LA DIRECCION DEL S...,DIRECCION SERVICIO DE SALUD,IQUIQUE,POINT (-70.15232 -20.21512)
1,PRAIS (S.S. IQUIQUE),PROGRAMA DE REPARACION Y ATENCION INTEGRAL DE ...,IQUIQUE,POINT (-70.15232 -20.21512)
2,UNIDAD DE SALUD FUNCIONARIO S.S. IQUIQUE,UNIDAD DE SALUD FUNCIONARIOS,IQUIQUE,POINT (-70.15232 -20.21512)
3,HOSPITAL DR. ERNESTO TORRES GALDAMES (IQUIQUE),HOSPITAL,IQUIQUE,POINT (-70.13893 -20.21435)
4,CLINICA IQUIQUE,CLINICA,IQUIQUE,POINT (-70.15497 -20.2183)


In [13]:
educacion_df.head()

,AGNO,RBD,DGV_RBD,NOM_RBD,TIPO_SOST,COD_REG_RB,NOM_REG_RB,COD_PRO_RB,COD_COM_RB,NOM_COM_RB,...,MAT_MHC_AD,MAT_MTP_RE,MAT_MTP_AD,MAT_TOTAL,MAT_HOM_TO,MAT_MUJ_TO,MAT_SI_TOT,CUR_SIM_TO,CUR_COMB_T,geometry
0,2021.0,8484.0,0.0,ESCUELA PUERTO TORO,1.0,12.0,MAG,122.0,12201.0,CABO DE HORNOS,...,0.0,0.0,0.0,5.0,4.0,1.0,0.0,4.0,0.0,POINT (-67.07622 -55.08317)
1,2021.0,8483.0,2.0,LICEO BICENTENARIO DONALD MC-INTYRE GRIFFITHS,1.0,12.0,MAG,122.0,12201.0,CABO DE HORNOS,...,0.0,0.0,0.0,424.0,212.0,212.0,0.0,15.0,0.0,POINT (-67.60534 -54.9348)
2,2021.0,24341.0,8.0,ESCUELA PAMPA GUANACO,1.0,12.0,MAG,123.0,12303.0,TIMAUKEL,...,0.0,0.0,0.0,5.0,0.0,5.0,0.0,0.0,1.0,POINT (-68.80295 -54.05223)
3,2021.0,8452.0,2.0,ESCUELA DE PUERTO HARRIS,1.0,12.0,MAG,121.0,12101.0,PUNTA ARENAS,...,0.0,0.0,0.0,52.0,29.0,23.0,0.0,0.0,2.0,POINT (-70.45337 -53.835)
4,2021.0,8482.0,4.0,ESCUELA IGNACIO CARRERA PINTO,1.0,12.0,MAG,123.0,12303.0,TIMAUKEL,...,0.0,0.0,0.0,20.0,11.0,9.0,0.0,2.0,2.0,POINT (-69.64666 -53.63903)


In [14]:
educacion_df = educacion_df[['NOM_RBD','NOM_COM_RB', 'geometry']]
educacion_df.reset_index(drop=True, inplace=True)
educacion_df.rename(columns={'NOM_RBD': 'NOMBRE_ESTABLECIMIENTO', 'NOM_COM_RB': 'COMUNA'}, inplace=True)
educacion_df

,NOMBRE_ESTABLECIMIENTO,COMUNA,geometry
0,ESCUELA PUERTO TORO,CABO DE HORNOS,POINT (-67.07622 -55.08317)
1,LICEO BICENTENARIO DONALD MC-INTYRE GRIFFITHS,CABO DE HORNOS,POINT (-67.60534 -54.9348)
2,ESCUELA PAMPA GUANACO,TIMAUKEL,POINT (-68.80295 -54.05223)
3,ESCUELA DE PUERTO HARRIS,PUNTA ARENAS,POINT (-70.45337 -53.835)
4,ESCUELA IGNACIO CARRERA PINTO,TIMAUKEL,POINT (-69.64666 -53.63903)
...,...,...,...
11280,ESCUELA HUMAPALCA,GENERAL LAGOS,POINT (-69.70278 -17.83439)
11281,ESCUELA COSAPILLA,GENERAL LAGOS,POINT (-69.42124 -17.76635)
11282,ESCUELA GUACOYO,GENERAL LAGOS,POINT (-69.34495 -17.76013)
11283,ESCUELA CHISLLUMA,GENERAL LAGOS,POINT (-69.6883 -17.69677)


In [15]:
parvularia_df.head()

,AGNO,ID_ESTAB,ORIGEN,NOM_ESTAB,COD_REG_ES,COD_PRO_ES,COD_COM_ES,NOM_COM_ES,COD_DEPROV,NOM_DEPROV,...,NT2_SI,NT2_H,NT2_M,N_TOTAL,DIRECCION,NUMERO,REFERENCIA,LATITUD,LONGITUD,geometry
0,2021.0,2.0,1.0,PARVULARIO LAS ESPIGUITAS,15.0,151.0,15101.0,ARICA,151.0,ARICA,...,0.0,41.0,43.0,170.0,RONALD ROSS,2165.0,POB. JUAN NOE,-18.469862,-70.302757,POINT (-70.30276 -18.46986)
1,2021.0,8.0,1.0,COLEGIO INTEGRADO EDUARDO FREI MONTALVA,15.0,151.0,15101.0,ARICA,151.0,ARICA,...,0.0,23.0,9.0,60.0,SAN MARCOS,611.0,ESQUINA GENERAL LAGOS,-18.482491,-70.317581,POINT (-70.31758 -18.48249)
2,2021.0,9.0,1.0,ESCUELA REPUBLICA DE ISRAEL,15.0,151.0,15101.0,ARICA,151.0,ARICA,...,0.0,52.0,55.0,190.0,MAGALLANES,1860.0,ENTRE SIMÓN BOLÍVAR Y BORGOÑO POBLACIÓN MAGIS...,-18.486691,-70.304520,POINT (-70.30452 -18.48669)
3,2021.0,10.0,1.0,ESCUELA REPUBLICA DE FRANCIA,15.0,151.0,15101.0,ARICA,151.0,ARICA,...,0.0,4.0,3.0,15.0,MARIA ILIA DEL PINO,728.0,INTERSECCIÓN DE CALLE SANTA MARÍA CON MARÍA IL...,-18.474369,-70.309036,POINT (-70.30904 -18.47437)
4,2021.0,11.0,1.0,ESC. GRAL. PEDRO LAGOS MARCHANT,15.0,151.0,15101.0,ARICA,151.0,ARICA,...,0.0,8.0,9.0,31.0,AVENIDA R SOTOMAYOR,1639.0,EN EL CERRO LA CRUZ,-18.491011,-70.311577,POINT (-70.31158 -18.49101)


In [16]:
parvularia_df = parvularia_df[['NOM_ESTAB','NOM_COM_ES', 'geometry']]
parvularia_df.reset_index(drop=True, inplace=True)
parvularia_df.rename(columns={'NOM_ESTAB': 'NOMBRE_ESTABLECIMIENTO', 'NOM_COM_ES': 'COMUNA'}, inplace=True)
parvularia_df

,NOMBRE_ESTABLECIMIENTO,COMUNA,geometry
0,PARVULARIO LAS ESPIGUITAS,ARICA,POINT (-70.30276 -18.46986)
1,COLEGIO INTEGRADO EDUARDO FREI MONTALVA,ARICA,POINT (-70.31758 -18.48249)
2,ESCUELA REPUBLICA DE ISRAEL,ARICA,POINT (-70.30452 -18.48669)
3,ESCUELA REPUBLICA DE FRANCIA,ARICA,POINT (-70.30904 -18.47437)
4,ESC. GRAL. PEDRO LAGOS MARCHANT,ARICA,POINT (-70.31158 -18.49101)
...,...,...,...
11946,MINAS DEL PRADO,COIHUECO,POINT (-71.70676 -36.66736)
11947,EL CAMARONCITO,ÑIQUÉN,POINT (-71.81751 -36.2827)
11948,PULGARCITO,SAN FABIÁN,POINT (-71.54871 -36.55738)
11949,MANZANITO,SAN NICOLÁS,POINT (-72.08572 -36.53958)


### Unimos los dataframes educacion y parvularia para tener un único dataframe de educación

In [17]:
educa_all = pd.concat([educacion_df, parvularia_df], ignore_index=True) 
educa_all

,NOMBRE_ESTABLECIMIENTO,COMUNA,geometry
0,ESCUELA PUERTO TORO,CABO DE HORNOS,POINT (-67.07622 -55.08317)
1,LICEO BICENTENARIO DONALD MC-INTYRE GRIFFITHS,CABO DE HORNOS,POINT (-67.60534 -54.9348)
2,ESCUELA PAMPA GUANACO,TIMAUKEL,POINT (-68.80295 -54.05223)
3,ESCUELA DE PUERTO HARRIS,PUNTA ARENAS,POINT (-70.45337 -53.835)
4,ESCUELA IGNACIO CARRERA PINTO,TIMAUKEL,POINT (-69.64666 -53.63903)
...,...,...,...
23231,MINAS DEL PRADO,COIHUECO,POINT (-71.70676 -36.66736)
23232,EL CAMARONCITO,ÑIQUÉN,POINT (-71.81751 -36.2827)
23233,PULGARCITO,SAN FABIÁN,POINT (-71.54871 -36.55738)
23234,MANZANITO,SAN NICOLÁS,POINT (-72.08572 -36.53958)


---

# Filtramos datos para cada ciudad (En base a AOI)

### Reproyección de los GeoDataFrames al mismo sistema de coordenadas

In [18]:
bomberos_df = bomberos_df.to_crs(aoi_arica.crs)
carabineros_df = carabineros_df.to_crs(aoi_arica.crs)
salud_df = salud_df.to_crs(aoi_arica.crs)
educacion_df = educacion_df.to_crs(aoi_arica.crs)
parvularia_df = parvularia_df.to_crs(aoi_arica.crs)

### Servicios de concepcion

In [19]:
bomberos_conce = bomberos_df[bomberos_df.geometry.intersects(aoi_conce.geometry.iloc[0])]
carabineros_conce = carabineros_df[carabineros_df.geometry.intersects(aoi_conce.geometry.iloc[0])]
salud_conce = salud_df[salud_df.geometry.intersects(aoi_conce.geometry.iloc[0])]
educa_all_conce = educa_all[educa_all.geometry.intersects(aoi_conce.geometry.iloc[0])]

### Servicios de Arica

In [20]:
bomberos_arica = bomberos_df[bomberos_df.geometry.intersects(aoi_arica.geometry.iloc[0])]
carabineros_arica = carabineros_df[carabineros_df.geometry.intersects(aoi_arica.geometry.iloc[0])]
salud_arica = salud_df[salud_df.geometry.intersects(aoi_arica.geometry.iloc[0])]
educa_all_arica = educa_all[educa_all.geometry.intersects(aoi_arica.geometry.iloc[0])]

### Servicios de La Serena

In [21]:
bomberos_serena = bomberos_df[bomberos_df.geometry.intersects(aoi_serena.geometry.iloc[0])]
carabineros_serena = carabineros_df[carabineros_df.geometry.intersects(aoi_serena.geometry.iloc[0])]
salud_serena = salud_df[salud_df.geometry.intersects(aoi_serena.geometry.iloc[0])]
educa_all_serena = educa_all[educa_all.geometry.intersects(aoi_serena.geometry.iloc[0])]

### Servicios de viña del mar

In [22]:
bomberos_vina = bomberos_df[bomberos_df.geometry.intersects(aoi_vina.geometry.iloc[0])]
carabineros_vina = carabineros_df[carabineros_df.geometry.intersects(aoi_vina.geometry.iloc[0])]
salud_vina = salud_df[salud_df.geometry.intersects(aoi_vina.geometry.iloc[0])]
educa_all_vina = educa_all[educa_all.geometry.intersects(aoi_vina.geometry.iloc[0])]

### Servicios de Valparaíso

In [23]:
bomberos_valpo = bomberos_df[bomberos_df.geometry.intersects(aoi_valpo.geometry.iloc[0])]
carabineros_valpo = carabineros_df[carabineros_df.geometry.intersects(aoi_valpo.geometry.iloc[0])]
salud_valpo = salud_df[salud_df.geometry.intersects(aoi_valpo.geometry.iloc[0])]
educa_all_valpo = educa_all[educa_all.geometry.intersects(aoi_valpo.geometry.iloc[0])]


### Servicios de rancagua

In [24]:
bomberos_rancagua = bomberos_df[bomberos_df.geometry.intersects(aoi_rancagua.geometry.iloc[0])]
carabineros_rancagua = carabineros_df[carabineros_df.geometry.intersects(aoi_rancagua.geometry.iloc[0])]
salud_rancagua = salud_df[salud_df.geometry.intersects(aoi_rancagua.geometry.iloc[0])]
educa_all_rancagua = educa_all[educa_all.geometry.intersects(aoi_rancagua.geometry.iloc[0])]

### Servicios de Puerto Montt

In [25]:
bomberos_pmontt = bomberos_df[bomberos_df.geometry.intersects(aoi_pmontt.geometry.iloc[0])]
carabineros_pmontt = carabineros_df[carabineros_df.geometry.intersects(aoi_pmontt.geometry.iloc[0])]
salud_pmontt = salud_df[salud_df.geometry.intersects(aoi_pmontt.geometry.iloc[0])]
educa_all_pmontt = educa_all[educa_all.geometry.intersects(aoi_pmontt.geometry.iloc[0])]

### Servicios de Valdivia

In [26]:
bomberos_valdivia = bomberos_df[bomberos_df.geometry.intersects(aoi_valdivia.geometry.iloc[0])]
carabineros_valdivia = carabineros_df[carabineros_df.geometry.intersects(aoi_valdivia.geometry.iloc[0])]
salud_valdivia = salud_df[salud_df.geometry.intersects(aoi_valdivia.geometry.iloc[0])]
educa_all_valdivia = educa_all[educa_all.geometry.intersects(aoi_valdivia.geometry.iloc[0])]

---

# Visualización de datos

## Concepción y sus servicios básicos

Utilicé ChatGPT. Le pregunté como agregar leyenda un mapa hecho con folium. 

Me respondió: Sí, puedes agregar una leyenda personalizada a tu mapa de folium, aunque no hay una función directa integrada para eso. Lo que se hace es agregar una etiqueta HTML flotante en una esquina del mapa con los colores y significados.

In [27]:
legend_html = """
<div style='position: fixed; 
     bottom: 30px; left: 30px; width: 200px; height: 130px; 
     border:2px solid grey; z-index:9999; font-size:14px;
     background-color: white; padding: 10px'>
<b>Leyenda</b><br>
<i style="background:blue; width:10px; height:10px; float:left; margin-right:5px"></i> Centros Educacionales<br>
<i style="background:purple; width:10px; height:10px; float:left; margin-right:5px"></i> Centros de Salud<br>
<i style="background:green; width:10px; height:10px; float:left; margin-right:5px"></i> Carabineros<br>
<i style="background:red; width:10px; height:10px; float:left; margin-right:5px"></i> Bomberos
</div>
"""

In [28]:
# Centro del mapa
centro_conce = aoi_conce.geometry.centroid.iloc[0].coords[0][::-1]

# Crear mapa con folium
m = folium.Map(location=centro_conce, zoom_start=13)

# Añadir AOI como capa
folium.GeoJson(
    aoi_conce,
    name='AOI - CONCEPCIÓN',
    style_function=lambda x: {'fillColor': 'gray', 'color': 'black', 'fillOpacity': 0.1}
).add_to(m)

# Establecimientos educacionales
folium.GeoJson(
    educa_all_conce,
    name='Establecimientos Educacionales',
    tooltip=folium.GeoJsonTooltip(
        fields=['NOMBRE_ESTABLECIMIENTO'],
        aliases=['Nombre:']
    ),
    marker=folium.CircleMarker(radius=2, color="blue", fill=True)
).add_to(m)

# Centros de salud
folium.GeoJson(
    salud_conce,
    name='Centros de Salud',
    tooltip=folium.GeoJsonTooltip(
        fields=['NOMBRE_ESTABLECIMIENTO'],
        aliases=['Nombre:']
    ),
    marker=folium.CircleMarker(radius=2, color="purple", fill=True)
).add_to(m)

# Cuarteles de carabineros
folium.GeoJson(
    carabineros_conce,
    name='Cuarteles de Carabineros',
    tooltip=folium.GeoJsonTooltip(
        fields=['NOMBRE_ESTABLECIMIENTO'],
        aliases=['Nombre:']
    ),
    marker=folium.CircleMarker(radius=2, color="green", fill=True)
).add_to(m)

# Cuarteles de bomberos
folium.GeoJson(
    bomberos_conce,
    name='Cuarteles de Bomberos',
    tooltip=folium.GeoJsonTooltip(
        fields=['NOMBRE_CUERPO'],
        aliases=['Nombre:']
    ),
    marker=folium.CircleMarker(radius=2, color="red", fill=True)
).add_to(m)

# Control de capas
folium.LayerControl().add_to(m)

m.get_root().html.add_child(folium.Element(legend_html))

# Mostrar mapa
m


# Fuentes
* Geolocalización
    *  Cuerpos de bomberos de chile (2017): [Datos Gob](https://datos.gob.cl/dataset/categoria-geoespacial-instalaciones-y-edificaciones/resource/942164c9-6538-478b-b73c-2672691b36c9)

    * Cuarteles de Carabineros de Chile (2018): [Geoportal](https://www.geoportal.cl/geoportal/catalog/35003/Cuarteles%20de%20Carabineros)

    * Establecimientos de salud (2024): [Geoportal](https://www.geoportal.cl/geoportal/catalog/35499/Establecimientos%20de%20salud%20de%20Chile)

    * Establecimientos de educación escolar (2022): [Geoportal](https://www.geoportal.cl/geoportal/catalog/35408/Establecimientos%20Educaci%C3%B3n%20Escolar)

    * Establecimientos de educación parvularia (2022): [Geoportal](https://www.geoportal.cl/geoportal/catalog/35553/Establecimientos%20Educaci%C3%B3n%20Parvularia)
